## 8. 데이터 준비하기: 조인, 병합, 변형

8.1 계층적 색인

In [1]:
import numpy as np
import pandas as pd

In [2]:
from IPython.display import display_html

In [ ]:
# 9개의 랜덤값 담은 시리즈 생성
# uniform : 균일분포. 0~1 사이에서 동일한 확률로 난수 생성
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

In [ ]:
# 시리즈의 인덱스 확인
# 멀티 인덱스
data.index

In [ ]:
# 인덱스로 값 조회
# data["b"]
# 슬라이스로 b부터 c까지 선택
data["b":"c"]
# b와 d만 선택
data.loc[["b", "d"]]

In [ ]:
# loc : 라벨로 값을 찾는 함수
# loc(행, 열)
# 멀티인덱스의 두번째 숫자 부분에서 2만 선택
data.loc[:, 2]

In [ ]:
# 멀티 인덱스를 가진 시리즈를 데이터프레임으로 변환
# 인덱스 중 숫자를 열로 이동
data.unstack()

In [ ]:
# 다시 원복
data.unstack().stack()

In [ ]:
# 0부터 11까지 숫자를 4행 3열로 변환하여 데이터프레임 생성
# 행은 멀티 인덱스 (알파벳과 숫자)
# 열도 멀티 인덱스 (지역과 색)
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["서울", "서울", "인천"],
                              ["Green", "Red", "Green"]])
frame

In [ ]:
# 각 인덱스에 이름 설정
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

In [ ]:
# 인덱스가 몇 단계인지 확인
frame.index.nlevels

In [ ]:
# 열의 첫번째 인덱스(지역)을 서울로 선택
frame["서울"]

8.1.1 계층의 순서를 바꾸고 정렬하기

In [ ]:
# 행 인덱스 순서 바꾸기 (문자,숫자 -> 숫자,문자)
# 인덱스만 바뀌고 값은 그대로
frame.swaplevel("key1", "key2")

In [ ]:
# 행 인덱스를 보기 좋게 정렬
# level=0 옵션 -> 첫 인덱스 단계(숫자)
frame.swaplevel(0, 1).sort_index(level=0)

8.1.2 계층별 요약 통계

In [ ]:
# 행 중에서 숫자 인덱스를 기준으로 그룹화 하여 합 구하기
# 같은 숫자끼리 묶기
frame.groupby(level="key2").sum()
# 열 중에서 color를 기준으로 합 구하기
# 같은 색끼리 묶기
frame.groupby(level="color", axis="columns").sum()

8.1.3 DataFrame의 열 사용하기

In [ ]:
# a,b,c,d 열을 담은 데이터 프레임 생성
frame = pd.DataFrame({"a": range(7), #0~6
                      "b": range(7, 0, -1), # 7~1
                      "c": ["one", "one", "one", "two", "two", "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
# c,d열을 행 인덱스로 변경
frame2 = frame.set_index(["c", "d"])
frame2

In [ ]:
# drop=False 옵션 -> c,d열을 행 인덱스로 지정하면서, 원래 열도 유지
frame.set_index(["c", "d"], drop=False)

In [ ]:
# 원래대로 원복
frame2.reset_index()

8.2 데이터 합치기

8.2.1 데이터베이스 스타일로 DataFrame 합치기

In [3]:
# key, data1 열을 담은 데이터 프레임 생성
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
# key, data2 열을 담은 데이터 프레임 생성
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df1
df2

display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        df1.to_html(), df2.to_html()
    ), raw=True)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
,key,data2
0,a,0
1,b,1


In [4]:
# key를 기준으로 inner join
# 두 프레임에 공통된 값만 남고, 나머지는 삭제됨
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [ ]:
# 명시적으로 공통키 설정하기
# on="key" 설정을 쓰지않아도, 두 프레임에 같은 이름의 열이 있으면 자동으로 기준이 됨
pd.merge(df1, df2, on="key")

In [6]:
# 1key, data1 열을 담은 프레임 생성
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
# rkey, data2 열을 담은 프레임 생성
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df3
df4
display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        df3.to_html(), df4.to_html()
    ), raw=True)

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
,rkey,data2
0,a,0
1,b,1


In [7]:
# 두 프레임에서 같은 이름의 열이 없을 때는
# 각각 기준이 되는 열을 지정해야함
# df3에서는 lkey, df4에서는 rkey를 기준으로 join
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [8]:
# outer join : 공통되지 않은 키도 포함
# 일치하지 않는 부분은 na로 채워짐
pd.merge(df1, df2, how="outer")
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [11]:
# key, data1 열을 담은 프레임 생성
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})
# key, data2 열을 담은 프레임 생성
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df1
df2
display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        df1.to_html(), df2.to_html()
    ), raw=True)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5
,key,data2
0,a,0
1,b,1
2,a,2


In [12]:
# left join : 왼쪽(df1)의 모든 key를 유지
# 왼쪽에는 있지만 오른쪽에 없는 c는 남고, 빈부분은 nan
# 오른쪽(df2)에 매칭되는 값이 없으면 NaN으로 채움
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [13]:
# inner join : 양쪽에 모두 존재하는 key만
# key가 중복되면 모든 조합으로 매칭됨
# df1에 b가 3개, df2에 b가 2개 ->결과에 b는 3*2 = 6개 행 생성
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [14]:
# key1,key2,lval 열을 담은 프레임 생성
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
# key1,key2,rval 열을 담은 프레임 생성
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        left.to_html(), right.to_html()
    ), raw=True)
# 두 개의 컬럼을 기준으로 join
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3
,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [15]:
# key1 컬럼을 기준으로 join
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


8.2.2 색인 병합하기

In [20]:
# key,value 열을 담은 프레임 생성
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
# group_val 열을 담은 프레임 생성
# 행 인덱스는 a,b
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
left1
right1
display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        left1.to_html(), right1.to_html()
    ), raw=True)
# left1에서는 key컬럼, right1는 인덱스를 기준으로 조인
pd.merge(left1, right1, left_on="key", right_index=True)

key 
 value 
 
 
 
 
 0 
 a 
 0 
 
 
 1 
 b 
 1 
 
 
 2 
 a 
 2 
 
 
 3 
 a 
 3 
 
 
 4 
 b 
 4 
 
 
 5 
 c 
 5 
 
 
 
 
 
 
 group_val 
 
 
 
 
 a 
 3.5 
 
 
 b 
 7.0

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [21]:
# outer join은 양쪽 프레임에서 모든 값을 가져옴
# 매치되지 않는 부분은 nan으로 채움
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [23]:
# key1,key2,data 열을 담은 프레임 생성
lefth = pd.DataFrame({"key1": ["서울", "서울", "서울", "인천", "인천"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
# 멀티 인덱스 생성 (key1=지역, key2=연도)
righth_index = pd.MultiIndex.from_arrays(
    [
        ["인천", "인천", "서울", "서울", "서울", "서울"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)
# 멀티 인덱스를 갖는 프레임 생성
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})
lefth
righth
display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        lefth.to_html(), righth.to_html()
    ), raw=True)

key1 
 key2 
 data 
 
 
 
 
 0 
 서울 
 2000 
 0 
 
 
 1 
 서울 
 2001 
 1 
 
 
 2 
 서울 
 2002 
 2 
 
 
 3 
 인천 
 2001 
 3 
 
 
 4 
 인천 
 2002 
 4 
 
 
 
 
 
 
 
 event1 
 event2 
 
 
 
 
 인천 
 2001 
 0 
 1 
 
 
 2000 
 2 
 3 
 
 
 서울 
 2000 
 4 
 5 
 
 
 2000 
 6 
 7 
 
 
 2001 
 8 
 9 
 
 
 2002 
 10 
 11

In [25]:
# lefth에서는 key1,key2 컬럼, righth에서는 인덱스를 기준으로 조인
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)
# outer join
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
0,서울,2000,0,4,5
0,서울,2000,0,6,7
1,서울,2001,1,8,9
2,서울,2002,2,10,11
4,인천,2000,<NA>,2,3
3,인천,2001,3,0,1
4,인천,2002,4,<NA>,<NA>


In [29]:
# 행인덱스는 a,c,e 열은 서울,인천을 가지는 프레임 생성
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["서울", "인천"]).astype("Int64")

# 행인덱스는 b,c,d,e 열은 부산,제주도를 가지는 프레임 생성
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["부산", "제주도"]).astype("Int64")
left2
right2
display_html(
    '<table><tr><td>{}</td><td>{}</td></tr></table>'.format(
        left2.to_html(), right2.to_html()
    ), raw=True)

# left2에서는 행인덱스(a,c,e) right2에서는 행인덱스(b,c,d,e)을 기준으로 조인
# outer join은 모든 값을 가져옴. 매치되는 부분이 없으면 nan으로 채움
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,서울,인천
a,1,2
c,3,4
e,5,6
,부산,제주도
b,7,8
c,9,10
d,11,12
e,13,14


,서울,인천,부산,제주도
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [30]:
# join은 기준컬럼이 없으면 행인덱스를 사용
left2.join(right2, how="outer")

,서울,인천,부산,제주도
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [31]:
# join은 join방식을 지정하지 않으면 기본값이 'left'
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


8.2.3 축 따라 이어 붙이기

In [33]:
# 0부터 11까지 숫자를 3행 4열 배열로 생성
arr = np.arange(12).reshape((3, 4))
arr
# 배열을 열방향으로 이어붙임
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [42]:
# 인덱스가 서로 다른 시리즈 3개 생성
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

In [45]:
s1
s2
s3
# 시리즈 3개 연결
# 위에서 아래로 행방향으로 연결됨
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [46]:
# axis="columns" -> 열방향으로 연결하기
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [52]:
# 두 시리즈를 행 방향으로 연결
s4 = pd.concat([s1, s3])
s4
# 두 시리즈를 열 방향으로 연결
# outer join 방식 사용
pd.concat([s1, s4], axis="columns")
# inner join 방식 사용
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [56]:
# 멀티 인덱스 설정
# 첫번째 : "one", "two", "three"
# 두번째 : a, b, f, g
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [57]:
# 열방향으로 조인하면 key는 열의 이름이됨
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [61]:
# 0부터 5까지 숫자를 3행 2열 배열로 생성
# 행 인덱스 : "a", "b", "c"
# 열 이름 : "one", "two"
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
# 5부터 9까지 숫자를 2행 2열 배열로 생성
# 행 인덱스 : "a", "c"
# 열 이름 : "three", "four"
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df1
df2
# 열 방향으로 연결
# 열에 멀티 인덱스 설정
# 첫번쩨 인덱스는 "level1", "level2"
# 두번째 인덱스는 "one", "two", "three", "four"
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [63]:
# 정규분포(-3~3) 난수로 3행 4열 배열 생성
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df1
df2

,b,d,a
0,0.035918,1.097438,-0.549878
1,-0.156178,-0.445064,0.558632


In [64]:
# 두 프레임을 세로 방향으로 연결하고
# 기존 인덱스는 버리고 새로운 인덱스를 할당
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.090896,0.251686,1.059764,-2.023513
1,0.772913,0.277172,-2.267267,-0.225874
2,-0.191671,0.088553,-0.680738,2.575688
3,-0.549878,0.035918,NaN,1.097438
4,0.558632,-0.156178,NaN,-0.445064


8.2.4 겹치는 데이터 합치기

In [66]:
# 2개의 시리즈 생성
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
a
b
# pd.isna(a) : a값이 nan이면 true, 값이 있으면 false
# where : 조건이 true이면 b, 아니면 a를 선택
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [68]:
# a가 있으면 a를 쓰고 없으면 b를 씀
# combine_first은 where과 달리 인덱스가 유지됨
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [ ]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

8.3 피벗

8.3.1

In [70]:
# 0부터 5까지 숫자를 2행 3열 배열로 생성
# 행 인덱스 : 서울, 인천
# 열 이름: one, two, three (이름은 number)
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["서울", "인천"], name="city"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
city,,,
서울,0,1,2
인천,3,4,5


In [71]:
# 열을 행으로 피벗(회전)
# 피벗 : 프레임 재배치
result = data.stack()
result
# 멀티 인덱스가 됨 (city, number)

city  number
서울    one       0
      two       1
      three     2
인천    one       3
      two       4
      three     5
dtype: int64

In [72]:
# 행을 열로 피벗
result.unstack()

number,one,two,three
city,,,
서울,0,1,2
인천,3,4,5


In [73]:
# 행을 열로 피벗할때, 멀티인덱스에서 단계를 지정
# 행 인덱스 (city,number) 중에서 제일 앞에 있는 city를 기준으로 피벗
result.unstack(level=0)
# 인덱스 코드 또는 인덱스 이름으로 지정
result.unstack(level="city")

city,서울,인천
number,,
one,0,3
two,1,4
three,2,5


In [76]:
# 2개의 시리즈 생성
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
s1
s2
# 시리즈를 세로 방향으로 연결하고 멀티 인덱스 설정
# key는 가장 바깥쪽 인덱스가 됨
# 1단계(one,two) 2단계(a,b,c,d,e)
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [81]:
# left,right 열을 담은 프레임 생성
# left열에 result 시리즈를 값으로 담기
# right 열에 reuslt 시리즈에 5를 더해서 값으로 담기
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df
# 행을 열로 피벗
# 열은 멀티 인덱스가 됨 (side, city)
df.unstack(level="city")

side   left    right    
city     서울 인천    서울  인천
number                  
one       0  3     5   8
two       1  4     6   9
three     2  5     7  10

In [82]:
# 열을 행으로 피벗
# side를 행으로 
df.unstack(level="city").stack(level="side")

C:\Users\imjiyeon\AppData\Local\Temp\ipykernel_254648\659390539.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level="city").stack(level="side")


city          서울  인천
number side         
one    left    0   3
       right   5   8
two    left    1   4
       right   6   9
three  left    2   5
       right   7  10

8.3.2 긴 형식에서 넓은 형식으로 피벗하기

In [93]:
# csv 읽기
data = pd.read_csv("examples/macrodata.csv")
# loc(행:열) -> year, quarter, realgdp, infl, unemp 열만 선택
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
# 상위 5개 행만 선택
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


In [94]:
# 연도와 분기를 조합하여 인덱스 생성
# 인덱스 이름은 date
# 예: 1959(연도) + 1(분기) -> 1959Q1
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
# periods
# 분기를 실제 날짜로 변경
# 예: 1959Q1 -> 1959-01-0
# 예: 1959Q2 -> 1959-04-01
data.index = periods.to_timestamp("D")
# 상위 5개 행만 조회
data.head()

C:\Users\imjiyeon\AppData\Local\Temp\ipykernel_254648\1828891581.py:4: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),


,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [98]:
# 열 순서 변경
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data
# 열 인덱스 이름을 item으로 지정
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [104]:
# data.stack() : realgdp, infl, unemp 열을 행으로 피벗
# reset_index() : 멀티 인덱스(date, item)를 일반 열로 바꿈
# rename : 데이터가 있는 열의 이름을 value로 설정
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))
long_data

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [105]:
# 처음부터 9번 행까지 슬라이스
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [106]:
# 행인덱스로 사용할 컬럼: date 
# 열로 사용할 컬럼: item
# 값으로 사용할 컬럼: value
# 이런 형태로 데이터 재배치
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [108]:
# 피벗시 value를 생략하면 열이 멀티인덱스가됨
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()
# pivoted["value"].head()

value                
item        infl   realgdp unemp
date                            
1959-01-01  0.00  2710.349   5.8
1959-04-01  2.34  2778.801   5.1
1959-07-01  2.74  2775.488   5.3
1959-10-01  0.27  2785.204   5.6
1960-01-01  2.31  2847.699   5.2

In [109]:
# pivot은 set_index로 멀티인덱스를 만들고 unstack으로 열을 펼치는 것과 같다
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                
item        infl   realgdp unemp
date                            
1959-01-01  0.00  2710.349   5.8
1959-04-01  2.34  2778.801   5.1
1959-07-01  2.74  2775.488   5.3
1959-10-01  0.27  2785.204   5.6
1960-01-01  2.31  2847.699   5.2

8.3.3 넓은 형식에서 긴 형식으로 피벗하기

In [110]:
# key,A,B,C,열을 가진 프레임 생성
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [111]:
# key는 그대로 두고
# 나머지 열을 variable과 value로 만듬
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [112]:
# 행인덱스로 쓸 컬럼: key
# 열로 쓸 컬럼: variable
# 값으로 쓸 컬럼: value
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [113]:
# 행인덱스를 일반 컬럼으로 변경
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [114]:
# key는 그대로 두고
# A와B 열을 variable과 value로 만듬
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [115]:
# id_vars는 지정하지 않고 A,B,C 열을 녹여 variable과 value로 만듬
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9
